In [1]:
import os

In [2]:
%pwd

'd:\\aiproject\\mydeeplearning_project\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\aiproject\\mydeeplearning_project'

In [6]:
# dagshub 
# mlflow >> used to track ML training experiment

# model >> lr >> 0..01 >> acc
# model >> lr >> 0.001 >> acc

In [7]:
#  https://dagshub.com/surajkale99/mydeeplearning_project.mlflow

In [9]:
#!pip install dagshub

In [10]:
import dagshub
dagshub.init(repo_owner='surajkale99', repo_name='mydeeplearning_project', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

c:\Users\Admin\anaconda3\envs\py310\lib\site-packages\rich\live.py:256: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=65e6260f-67f6-4f5e-99be-32fd8f3926e7&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=021af923aca5681b02079a1d25f07deb77786a18ff9a04f84a72871407708303




Accessing as surajkale99

Initialized MLflow to track repo "surajkale99/mydeeplearning_project"

Repository surajkale99/mydeeplearning_project initialized!

c:\Users\Admin\anaconda3\envs\py310\lib\site-packages\mlflow\utils\requirements_utils.py:12: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [11]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/surajkale99/mydeeplearning_project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="surajkale99"
os.environ["MLFLOW_TRACKING_PASSWORD"]="e4f2406488322cebc687e5c389841eb67f8beeec"

In [13]:
mlflow.set_tracking_uri("https://dagshub.com/surajkale99/mydeeplearning_project.mlflow")

In [14]:
# entity 